<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C11-Offensive_Forensics/Vulnerability_Reconnaissance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Vulnerability Reconnaissance__*